In [116]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import rdDetermineBonds
from rdkit.Chem.Draw import IPythonConsole
from openff.toolkit import Molecule, Topology, ForceField
from openff.interchange import Interchange
from openff.units import unit
from openff.units.openmm import from_openmm, to_openmm
import openmm
import numpy as np
import sys
from pdb_wizard import PBC
import random
from copy import deepcopy
import scipy

In [117]:
mol = Chem.MolFromXYZFile('./HKUST-1.xyz')

rdDetermineBonds.DetermineConnectivity(mol)
# IPythonConsole.drawMol3D(mol)

In [118]:
editable = Chem.EditableMol(mol)
# No kill like over kill
METALS = [3, 4, 11, 12, 13, 19, 20, 21, 22, 23, 24, 25 ,26, 27, 28, 29, 30, 31, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]

for bond in mol.GetBonds():
    a1 = bond.GetBeginAtom()
    a1_idx = bond.GetBeginAtomIdx()
    a2 = bond.GetEndAtom()
    a2_idx = bond.GetEndAtomIdx()
    if a1.GetAtomicNum() in METALS or a2.GetAtomicNum() in METALS:
        editable.RemoveBond(a1_idx, a2_idx)
    
# IPythonConsole.drawMol3D(editable.GetMol())

In [119]:
# You could change sanitizeFrags to True, but I doubt it's necessary
frags = Chem.GetMolFrags(editable.GetMol(), asMols=True, sanitizeFrags=True)
for frag in frags:
    if frag.GetNumAtoms() == 1:
        charge = 2
    else:
        charge = -3
    rdDetermineBonds.DetermineBonds(frag, charge=charge)

In [120]:
mols = [Molecule.from_rdkit(frag) for frag in frags]

In [121]:
for mol in mols:
    if mol.to_smiles() == '[Cu]':
        mol.atom(0).formal_charge = 2

In [122]:
def append_system(existing_system, system_to_append, cutoff, index_map=None):
    """Appends a system object onto the end of an existing system.

    Parameters
    ----------
    existing_system: openmm.System, optional
        The base system to extend.
    system_to_append: openmm.System
        The system to append.
    cutoff: openff.evaluator.unit.Quantity
        The nonbonded cutoff
    index_map: dict of int and int, optional
        A map to apply to the indices of atoms in the `system_to_append`.
        This is predominantly to be used when the ordering of the atoms
        in the `system_to_append` does not match the ordering in the full
        topology.
    """
    supported_force_types = [
        openmm.HarmonicBondForce,
        openmm.HarmonicAngleForce,
        openmm.PeriodicTorsionForce,
        openmm.NonbondedForce,
        openmm.RBTorsionForce,
        openmm.CustomNonbondedForce,
        openmm.CustomBondForce,
    ]

    number_of_appended_forces = 0
    index_offset = existing_system.getNumParticles()

    # Create an index map if one is not provided.
    if index_map is None:
        index_map = {i: i for i in range(system_to_append.getNumParticles())}

    # Append the particles.
    for index in range(system_to_append.getNumParticles()):
        index = index_map[index]
        existing_system.addParticle(system_to_append.getParticleMass(index))

    # Append the constraints
    for index in range(system_to_append.getNumConstraints()):
        index_a, index_b, distance = system_to_append.getConstraintParameters(index)

        index_a = index_map[index_a]
        index_b = index_map[index_b]

        existing_system.addConstraint(
            index_a + index_offset, index_b + index_offset, distance
        )

    # Validate the forces to append.
    for force_to_append in system_to_append.getForces():
        if type(force_to_append) in supported_force_types:
            continue

        raise ValueError(
            f"The system contains an unsupported type of "
            f"force: {type(force_to_append)}."
        )

    # Append the forces.
    for force_to_append in system_to_append.getForces():
        existing_force = None

        for force in existing_system.getForces():
            if type(force) not in supported_force_types:
                raise ValueError(
                    f"The existing system contains an unsupported type "
                    f"of force: {type(force)}."
                )

            if type(force_to_append) is not type(force):
                continue

            if isinstance(
                force_to_append, openmm.CustomNonbondedForce
            ) or isinstance(force_to_append, openmm.CustomBondForce):
                if force_to_append.getEnergyFunction() != force.getEnergyFunction():
                    continue

            existing_force = force
            break

        if existing_force is None:
            if isinstance(force_to_append, openmm.CustomNonbondedForce):
                existing_force = openmm.CustomNonbondedForce(
                    force_to_append.getEnergyFunction()
                )
                existing_force.setCutoffDistance(cutoff)
                existing_force.setNonbondedMethod(
                    openmm.CustomNonbondedForce.CutoffPeriodic
                )
                for index in range(force_to_append.getNumGlobalParameters()):
                    existing_force.addGlobalParameter(
                        force_to_append.getGlobalParameterName(index),
                        force_to_append.getGlobalParameterDefaultValue(index),
                    )
                for index in range(force_to_append.getNumPerParticleParameters()):
                    existing_force.addPerParticleParameter(
                        force_to_append.getPerParticleParameterName(index)
                    )
                existing_system.addForce(existing_force)

            elif isinstance(force_to_append, openmm.CustomBondForce):
                existing_force = openmm.CustomBondForce(
                    force_to_append.getEnergyFunction()
                )
                for index in range(force_to_append.getNumGlobalParameters()):
                    existing_force.addGlobalParameter(
                        force_to_append.getGlobalParameterName(index),
                        force_to_append.getGlobalParameterDefaultValue(index),
                    )
                for index in range(force_to_append.getNumPerBondParameters()):
                    existing_force.addPerBondParameter(
                        force_to_append.getPerBondParameterName(index)
                    )
                existing_system.addForce(existing_force)

            else:
                existing_force = type(force_to_append)()
                existing_system.addForce(existing_force)

        if isinstance(force_to_append, openmm.HarmonicBondForce):
            # Add the bonds.
            for index in range(force_to_append.getNumBonds()):
                index_a, index_b, *parameters = force_to_append.getBondParameters(
                    index
                )

                index_a = index_map[index_a]
                index_b = index_map[index_b]

                existing_force.addBond(
                    index_a + index_offset, index_b + index_offset, *parameters
                )

        elif isinstance(force_to_append, openmm.HarmonicAngleForce):
            # Add the angles.
            for index in range(force_to_append.getNumAngles()):
                (
                    index_a,
                    index_b,
                    index_c,
                    *parameters,
                ) = force_to_append.getAngleParameters(index)

                index_a = index_map[index_a]
                index_b = index_map[index_b]
                index_c = index_map[index_c]

                existing_force.addAngle(
                    index_a + index_offset,
                    index_b + index_offset,
                    index_c + index_offset,
                    *parameters,
                )

        elif isinstance(force_to_append, openmm.PeriodicTorsionForce):
            # Add the torsions.
            for index in range(force_to_append.getNumTorsions()):
                (
                    index_a,
                    index_b,
                    index_c,
                    index_d,
                    *parameters,
                ) = force_to_append.getTorsionParameters(index)

                index_a = index_map[index_a]
                index_b = index_map[index_b]
                index_c = index_map[index_c]
                index_d = index_map[index_d]

                existing_force.addTorsion(
                    index_a + index_offset,
                    index_b + index_offset,
                    index_c + index_offset,
                    index_d + index_offset,
                    *parameters,
                )

        elif isinstance(force_to_append, openmm.NonbondedForce):
            # Add the vdW parameters
            for index in range(force_to_append.getNumParticles()):
                index = index_map[index]

                existing_force.addParticle(
                    *force_to_append.getParticleParameters(index)
                )

            # Add the 1-2, 1-3 and 1-4 exceptions.
            for index in range(force_to_append.getNumExceptions()):
                (
                    index_a,
                    index_b,
                    *parameters,
                ) = force_to_append.getExceptionParameters(index)

                index_a = index_map[index_a]
                index_b = index_map[index_b]

                existing_force.addException(
                    index_a + index_offset, index_b + index_offset, *parameters
                )

        elif isinstance(force_to_append, openmm.RBTorsionForce):
            # Support for RBTorisionForce needed for OPLSAA, etc
            for index in range(force_to_append.getNumTorsions()):
                torsion_params = force_to_append.getTorsionParameters(index)
                for i in range(4):
                    torsion_params[i] = index_map[torsion_params[i]] + index_offset

                existing_force.addTorsion(*torsion_params)

        elif isinstance(force_to_append, openmm.CustomNonbondedForce):
            for index in range(force_to_append.getNumParticles()):
                nb_params = force_to_append.getParticleParameters(index_map[index])
                existing_force.addParticle(nb_params)

            # Add the 1-2, 1-3 and 1-4 exceptions.
            for index in range(force_to_append.getNumExclusions()):
                (
                    index_a,
                    index_b,
                ) = force_to_append.getExclusionParticles(index)

                index_a = index_map[index_a]
                index_b = index_map[index_b]

                existing_force.addExclusion(
                    index_a + index_offset, index_b + index_offset
                )

        elif isinstance(force_to_append, openmm.CustomBondForce):
            for index in range(force_to_append.getNumBonds()):
                index_a, index_b, bond_params = force_to_append.getBondParameters(
                    index
                )

                index_a = index_map[index_a] + index_offset
                index_b = index_map[index_b] + index_offset

                existing_force.addBond(index_a, index_b, bond_params)

        number_of_appended_forces += 1

    if number_of_appended_forces != system_to_append.getNumForces():
        raise ValueError("Not all forces were appended.")

def create_empty_system():
    system = openmm.System()
    system.addForce(openmm.HarmonicBondForce())
    system.addForce(openmm.HarmonicAngleForce())
    system.addForce(openmm.PeriodicTorsionForce())

    nonbonded_force = openmm.NonbondedForce()
    nonbonded_force.setNonbondedMethod(openmm.NonbondedForce.NoCutoff)

    system.addForce(nonbonded_force)

    return system

In [123]:
TEMP = to_openmm(298 * unit.kelvin)
PRESSURE = to_openmm(1 * unit.atm)
TIMESTEP = to_openmm(2 * unit.femtosecond)
GAS = Molecule.from_smiles('O=C=O')
ATOMS_PER_GAS_MOLECULE = len(GAS.atoms)
GAS_POS = np.array([
    (np.array([0, 0, 0])),
    (np.array([0.1163, 0, 0])),
    (np.array([0.2326, 0, 0]))
])  # nm
BOLTZMANN = (openmm.unit.BOLTZMANN_CONSTANT_kB * openmm.unit.AVOGADRO_CONSTANT_NA)
OPENFF_FF = ForceField("openff-2.2.0-uff.offxml", load_plugins=True)
PROB_INSERT_DELETE = 0.333
R_CUTOFF = 0.1 * openmm.unit.nanometer


In [124]:
MOF_TOP = Topology.from_molecules(mols)   
BOX_SIZE = 2.6343  # nm 
MOF_PBC = PBC(BOX_SIZE, BOX_SIZE, BOX_SIZE, 90, 90, 90)
MOF_TOP.box_vectors = np.array([[BOX_SIZE, 0, 0], [0, BOX_SIZE, 0], [0, 0, BOX_SIZE]]) * unit.nanometer
MOF_TOP.is_periodic = True

interchange = Interchange.from_smirnoff(topology=MOF_TOP, force_field=OPENFF_FF)
MOF_OPENMM_SYS = interchange.to_openmm(combine_nonbonded_forces=False)
for i in range(MOF_OPENMM_SYS.getNumParticles()):
    MOF_OPENMM_SYS.setParticleMass(i, 0.0)

In [125]:
gas_mols = [deepcopy(GAS)]
gas_top = Topology.from_molecules(gas_mols)
interchange = Interchange.from_smirnoff(topology=gas_top, force_field=OPENFF_FF)
openmm_sys = interchange.to_openmm(combine_nonbonded_forces=False)

# openmm_sys.addForce(openmm.MonteCarloBarostat(PRESSURE, TEMP))
openmm_integrator = openmm.LangevinIntegrator(TEMP, 1, TIMESTEP)

openmm_sim = openmm.app.Simulation(gas_top.to_openmm(), openmm_sys, openmm_integrator, platform=openmm.Platform.getPlatformByName("CPU"))

openmm_sim.context.setPositions(to_openmm(GAS_POS * unit.nanometer))
openmm_sim.minimizeEnergy()
context = openmm_sim.context.getState(getEnergy=True)
GAS_FORMATION_ENERGY = context.getPotentialEnergy()

In [126]:
def monte_carlo_test(E_new, E_old):
    delta_E = E_new - E_old
    probability = np.exp(-delta_E / (BOLTZMANN * TEMP))
    random_number = random.uniform(0, 1)
    
    return probability >= random_number

def build_context(num_gases, positions) -> openmm.Context:
    mof_openmm_sys = deepcopy(MOF_OPENMM_SYS)

    openmm_integrator = openmm.LangevinIntegrator(TEMP, 1, TIMESTEP)

    if num_gases != 0:
        openff_top = deepcopy(MOF_TOP)
        gas_mols = [deepcopy(GAS) for _ in range(num_gases)]
        openff_top.add_molecules(gas_mols)
        interchange = Interchange.from_smirnoff(topology=openff_top, force_field=OPENFF_FF)
        openmm_sys = interchange.to_openmm(combine_nonbonded_forces=False)

        context = openmm.Context(openmm_sys, openmm_integrator, openmm.Platform.getPlatformByName("CPU"))
        context.setPositions(positions)
    else:
        context = openmm.Context(mof_openmm_sys, openmm_integrator, openmm.Platform.getPlatformByName("CPU"))
        context.setPositions(positions)
    
    return context

def context_energy(context: openmm.Context) -> float:
    context.getIntegrator().step(1)
    state = context.getState(getEnergy=True)
    return state.getPotentialEnergy()

def system_energy(gases: np.ndarray) -> float:
    mof_openmm_sys = deepcopy(MOF_OPENMM_SYS)
    positions = deepcopy(MOF_TOP.get_positions().m)
    
    if len(gases) != 0:
        gas_mols = [deepcopy(GAS) for _ in range(len(gases) // ATOMS_PER_GAS_MOLECULE)]
        gas_top = Topology.from_molecules(gas_mols)
        gas_interchange = Interchange.from_smirnoff(topology=gas_top, force_field=OPENFF_FF)
        gas_openmm_sys = gas_interchange.to_openmm(combine_nonbonded_forces=False)
        
        # Create modeller with MOF topology and positions
        modeller = openmm.app.Modeller(MOF_TOP.to_openmm(), to_openmm(positions * unit.nanometer))
        
        # Add gas topology and positions
        gas_positions = to_openmm([x.m for x in gases] * unit.nanometer)
        modeller.add(gas_top.to_openmm(), gas_positions)
        
        # Get merged topology and positions
        merged_top = modeller.topology
        merged_positions = modeller.positions
        
        # Combine force systems
        append_system(mof_openmm_sys, gas_openmm_sys, 1 * unit.nanometer)
        
        openmm_integrator = openmm.LangevinIntegrator(TEMP, 1, TIMESTEP)
        openmm_sim = openmm.app.Simulation(merged_top, mof_openmm_sys, openmm_integrator, 
                                         platform=openmm.Platform.getPlatformByName("CPU"))
        openmm_sim.context.setPositions(merged_positions)
    else:
        openmm_integrator = openmm.LangevinIntegrator(TEMP, 1, TIMESTEP)
        openmm_sim = openmm.app.Simulation(MOF_TOP.to_openmm(), mof_openmm_sys, openmm_integrator, 
                                         platform=openmm.Platform.getPlatformByName("CPU"))
        openmm_sim.context.setPositions(to_openmm(positions * unit.nanometer))
    
    pdb_reporter = openmm.app.PDBReporter('out.pdb', 1)
    openmm_sim.reporters.append(pdb_reporter)

    openmm_sim.minimizeEnergy()
    openmm_sim.step(1)
    
    context = openmm_sim.context.getState(getEnergy=True)
    return context.getPotentialEnergy()

def suggest_gas_position(current_positions):
    while True:
        new_pos = deepcopy(GAS_POS)
        rotation_matrix = scipy.spatial.transform.Rotation.random().as_matrix()
        shift = np.array([random.uniform(0, BOX_SIZE), random.uniform(0, BOX_SIZE), random.uniform(0, BOX_SIZE)])  # nm
        new_pos = np.dot(new_pos, rotation_matrix) + shift
        valid = True

        for atom in new_pos:
            for dim in atom:
                if dim < 0 or dim > BOX_SIZE:
                    valid = False

        for existing_atom in current_positions.value_in_unit(openmm.unit.nanometer):
            for new_atom in new_pos:
                if MOF_PBC.min_image((existing_atom - new_atom)) < R_CUTOFF.value_in_unit(openmm.unit.nanometer):
                    valid = False

        if not valid:
            continue

        return new_pos

In [139]:
random.seed(42)
old_energy = system_energy([])

gases = []  # Must be a list where elements are Quantity<[x, y, z] * unit.nanometer> coordinates
contexts = {0: build_context(0, MOF_TOP.get_positions().m)}
num_gases = 0

def get_context(num_gases: int, contexts: dict[int, openmm.Context]) -> openmm.Context:
    if num_gases not in contexts:
        if num_gases - 1 not in contexts:
            print("This function uses the next number smaller as the basis for the next context, and for some reason the context for the previous number of gases is not available.")
            assert num_gases - 1 in contexts

        previous_context = contexts[num_gases - 1]
        previous_state = previous_context.getState(getPositions=True)
        previous_positions = previous_state.getPositions(asNumpy=True)

        new_positions = deepcopy(previous_positions)
        new_gas_position = suggest_gas_position(new_positions)
        total_positions = np.vstack((new_positions.value_in_unit(openmm.unit.nanometer), new_gas_position))

        new_context = build_context(num_gases, total_positions)
        contexts[num_gases] = new_context
    return contexts[num_gases]

len(get_context(1, contexts).getState(getPositions=True).getPositions(asNumpy=True))

# for timestep in range(1000):
#     operation = random.random()
#     if operation < PROB_INSERT_DELETE:  # Insert
#         print('Inserting')
#         new_gases = deepcopy(gases)
#         new_pos = suggest_gas_position(new_gases)
#         new_gases.extend(new_pos)
#         E_new = system_energy(new_gases) - GAS_FORMATION_ENERGY
#     elif operation < 2 * PROB_INSERT_DELETE:  # Delete
#         if len(gases) == 0:
#             continue
#         print('Deleting')
#         gas_to_remove = random.randint(0, (len(gases) // ATOMS_PER_GAS_MOLECULE) - 1)
#         new_gases = deepcopy(gases)
#         for _ in range(ATOMS_PER_GAS_MOLECULE):
#             new_gases.pop(gas_to_remove)
#         E_new = system_energy(new_gases) + GAS_FORMATION_ENERGY
#     else:  # Translate
#         if len(gases) == 0:
#             continue
#         print('Translating')
#         new_gases = deepcopy(gases)
#         gas_to_translate = random.randint(0, (len(gases) // ATOMS_PER_GAS_MOLECULE) - 1)
#         new_pos = suggest_gas_position(new_gases)
#         new_gases[gas_to_translate * ATOMS_PER_GAS_MOLECULE : gas_to_translate * ATOMS_PER_GAS_MOLECULE + ATOMS_PER_GAS_MOLECULE] = new_pos
#         E_new = system_energy(new_gases)
#         pass

#     if monte_carlo_test(E_new, old_energy):
#         print('Accepted')
#         old_energy = E_new
#         gases = new_gases
        
#     print('Step: ', timestep)
#     print('Num gas molecules:', len(gases) // ATOMS_PER_GAS_MOLECULE)
#     print()


This function uses the next number smaller as the basis for the next context, and for some reason the context for the previous number of gases is not available.


AssertionError: 